<a href="https://colab.research.google.com/github/PinkalPawar/ANN-Architecture/blob/main/Sentiment_Analysis%20ProjectCV_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install gensim


In [ ]:
!pip install contractions

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 10.1 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import re
from gensim.parsing.preprocessing import remove_stopwords
import contractions
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.callbacks import EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#! pip install contractions

In [ ]:
df1=pd.read_csv("/content/drive/MyDrive/Gen-AI/Data_sets/IMDB Dataset.csv")

In [ ]:
df1.shape

(50000, 2)

In [ ]:
df1.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df1.isnull().sum()

,0
review,0
sentiment,0


In [ ]:
df1["review"][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

#PREPROCESSING

In [ ]:
html_pattern = re.compile(r"<[^>]+>")
remove_non_alpha = re.compile(r"[^a-zA-Z ]")

In [ ]:
def preprocessing_text(review):

  #lower
  review = review.lower()

  #removing html
  review = html_pattern.sub("", review)

  #removing stopwords
  review = remove_stopwords(review)

  # remove_non_alpha
  review =  remove_non_alpha.sub("", review)

  # remove contractions
  review=contractions.fix(review)

  return review

In [ ]:
#df["review"] = df["review"].apply(preprocessing_text)

In [ ]:
df1["review"] = df1["review"].apply(preprocessing_text)

In [ ]:
df1.sentiment.value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(split=" ")

tokenizer.fit_on_texts(df1["review"].values)

In [ ]:
# from gensim.models import Word2Vec
# sentences = [sent.split() for sent in df1["review"]]
# word2vec_model = Word2Vec(sentences, vector_size=100, window=5)
# len(word2vec_model.wv["movie"])

In [ ]:
# embedding_matrix = np.zeros((214487+1, 100)) # vocab_size, vector_size

# for word, index in tokenizer.word_index.items():
#   if word in word2vec_model.wv:
#     embedding_matrix[index] = word2vec_model.wv[word]
#   else:
#     embedding_matrix[index] = np.zeros(100) # can be random value vector


In [ ]:
import numpy as np
from gensim.models import FastText

# Assuming df1["review"] is a column in your dataframe containing the text data
sentences = [sent.split() for sent in df1["review"]]

# Train FastText model
fasttext_model = FastText(sentences, vector_size=100, window=5)

# Create the embedding matrix
vocab_size = 214487  # adjust this according to your actual vocabulary size
embedding_matrix = np.zeros((vocab_size + 1, 100))  # vocab_size + 1 for padding index 0

for word, index in tokenizer.word_index.items():
    if word in fasttext_model.wv:
        embedding_matrix[index] = fasttext_model.wv[word]
    else:
        embedding_matrix[index] = np.zeros(100)  # or use random values if preferred

# Example: len(fasttext_model.wv["movie"]) will return the dimension of the embedding
print(len(fasttext_model.wv["movie"]))


100


In [ ]:
# len(tokenizer.index_word)

In [ ]:
# tokenizer.index_word

In [ ]:
# len(tokenizer.index_word)

In [ ]:
X = tokenizer.texts_to_sequences(df1["review"].values)

In [ ]:
# X

In [ ]:
# len(X[0])

In [ ]:
# len(df1["review"][0].split())

In [ ]:
#x = pad_sequences(X,padding = "post")
x = pad_sequences(X)

In [ ]:
len(x[100])

1348

In [ ]:
y = np.array([1 if label=="positive" else 0 for label in df1["sentiment"]])

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [ ]:
len(y)

50000

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(x,y,random_state=42,test_size=0.2,stratify=y)

In [ ]:
import keras

In [ ]:
#input_dim=214487+1, output_dim=100, input_length=1348

In [ ]:
# model initialization
model = keras.Sequential([
    keras.layers.Embedding(input_dim=214487+1, weights=[embedding_matrix], input_length=1348, trainable=False,output_dim=100),
    keras.layers.Dropout(0.25),
    keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    # keras.layers.Dropout(0.25),  # Add dropout after the first LSTM layer
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    # keras.layers.Dropout(0.25),  # Add dropout after the second LSTM layer
    # keras.layers.Dense(24, activation='relu'),
    # keras.layers.Dropout(0.25),  # Add dropout after the dense layer
    keras.layers.Dense(1, activation='sigmoid')
])
# compile model
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
# model summary
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1348, 100)         21448800  
                                                                 
 dropout_9 (Dropout)         (None, 1348, 100)         0         
                                                                 
 bidirectional_8 (Bidirecti  (None, 1348, 128)         84480     
 onal)                                                           
                                                                 
 bidirectional_9 (Bidirecti  (None, 128)               98816     
 onal)                                                           
                                                                 
 dense_4 (Dense)             (None, 1)                 129       
                                                                 
Total params: 21632225 (82.52 MB)
Trainable params: 18

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [ ]:
model.fit(X_train, Y_train, batch_size=256, epochs=10, validation_split = 0.2)

Epoch 1/10
125/125 [==============================] - 47s 314ms/step - loss: 0.4456 - accuracy: 0.7956 - val_loss: 0.3761 - val_accuracy: 0.8441
Epoch 2/10
125/125 [==============================] - 37s 296ms/step - loss: 0.3564 - accuracy: 0.8481 - val_loss: 0.3208 - val_accuracy: 0.8644
Epoch 3/10
125/125 [==============================] - 39s 312ms/step - loss: 0.3354 - accuracy: 0.8607 - val_loss: 0.3109 - val_accuracy: 0.8683
Epoch 4/10
125/125 [==============================] - 37s 297ms/step - loss: 0.3192 - accuracy: 0.8681 - val_loss: 0.3178 - val_accuracy: 0.8637
Epoch 5/10
125/125 [==============================] - 37s 295ms/step - loss: 0.3076 - accuracy: 0.8705 - val_loss: 0.2934 - val_accuracy: 0.8704
Epoch 6/10
125/125 [==============================] - 39s 309ms/step - loss: 0.2889 - accuracy: 0.8798 - val_loss: 0.2965 - val_accuracy: 0.8739
Epoch 7/10
125/125 [==============================] - 37s 294ms/step - loss: 0.2835 - accuracy: 0.8804 - val_loss: 0.2938 - val_ac

In [ ]:
##TEST

In [ ]:
pred=model.predict(X_test)

313/313 [==============================] - 21s 64ms/step


In [ ]:
t1=[]
for i in pred:
  #print(i)
  if i >=0.5:
    t1.append(1)
  else:
    t1.append(0)


In [ ]:
len(t1)

10000

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(Y_test,t1)
accuracy_score(Y_test,t1)


0.8819

In [ ]:
"""model.fit(x, y,
          batch_size=2024,
          epochs=10,
          validation_split=0.2
          #class_weights=class_weight_dict,
#callbacks=EarlyStopping(monitor="val_loss", patience=4, min_delta=0.0001)
          )

SyntaxError: incomplete input (<ipython-input-146-c076de97df67>, line 1)

## TEST

In [ ]:
test=pd.read_csv("/content/test.csv")

In [ ]:
test["review"] = test["review"].apply(preprocessing_text)

In [ ]:
from keras.preprocessing.text import Tokenizer

tokenizer1 = Tokenizer(split=" ")

tokenizer1.fit_on_texts(test["review"].values)

In [ ]:
y1=tokenizer1.texts_to_sequences(test["review"].values)

In [ ]:
padded_review = pad_sequences(y1, maxlen=1348)


In [ ]:
len(padded_review[0])

In [ ]:
pred=model.predict(padded_review)

In [ ]:
len(pred)

In [ ]:
t1=[]
for i in pred:
  print(i)
  if i >=0.5:
    t1.append(1)
  else:
    t1.append(0)


In [ ]:
len(t1)

In [ ]:
t1

In [ ]:
act = np.array([1 if label=="positive" else 0 for label in test["sentiment"]])

In [ ]:
act

In [ ]:
t1

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(act,t1)
accuracy_score(act,t1)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
confusion_matrix(act,t1)